In [1]:
#import pandas
import os
import pandas as pd 

**Step 1 load data**

In [2]:
#load data
aisles = pd.read_csv('datasets_4931_7487_aisles.csv') 
departments = pd.read_csv('datasets_4931_7487_departments.csv')
products_prior = pd.read_csv('order_products__prior.csv')
products_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

Aisles df key id : aisle_id

In [3]:
#inspect data 
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


departments key id: department_id 

In [4]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [5]:
products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


products key id: order_id, product_id 
gives order added to cart, and if reordered or not 

In [6]:
products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


key id: order id, eval_set : gives which set the order is from, order_dow: day of week

In [7]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


key id : product id 

In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


**Step 2 merge dataframes**

In [9]:

product_info = products.merge(aisles, on = 'aisle_id',  how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes
2,102,Danish Butter Cookies,61,19,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes


In [10]:
product_info = product_info.merge(departments, on = 'department_id', how='inner')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [ ]:
#merge orders and products_prior : 

prior_orders = orders.merge(products_prior, on = 'order_id', how = 'inner')
prior_orders = prior_orders.merge(product_info, on = 'product_id', how = 'inner')
prior_orders.head()

In [ ]:
user = prior_orders.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()

In [ ]:
user_avg_reorder = prior_orders.groupby('user_id')['reordered'].mean().to_frame('avg_reorder')

user = user.merge(user_avg_reorder, on = 'user_id', how = 'inner')
user.head()

In [ ]:
product = prior_orders.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
product.head()

In [ ]:
product_reorder = prior_orders.groupby('product_id')['reordered'].sum().to_frame('p_total_reorders')
product = product.merge(product_reorder, on = 'product_id', how= 'inner')
product['percent_reorder'] = (product['p_total_reorders']/product['p_total_purchases'])
product.head()

In [ ]:
product = pd.merge(product, product_info, on='product_id', how ='inner')
product.head()

In [ ]:
product_sorted = product.sort_values(by= 'p_total_purchases', ascending = False)
product_sorted.head(15)

In [ ]:
# merge orders and products train:

train_orders = orders.merge(products_train, on = 'order_id', how = 'inner')
train_orders = train_orders.merge(product_info, on = 'product_id', how = 'inner')
train_orders.head()

**Step 3 Data Definition**

In [ ]:
prior_orders.info()

In [ ]:
train_orders.info()

In [ ]:
#change type to category
train_orders['aisle'] = train_orders['aisle'].astype('category')
train_orders['department'] = train_orders['department'].astype('category')
train_orders['eval_set'] = train_orders['eval_set'].astype('category')
train_orders['product_name'] = train_orders['product_name'].astype('category')
prior_orders['aisle'] = prior_orders['aisle'].astype('category')
prior_orders['department'] = prior_orders['department'].astype('category')
prior_orders['eval_set'] = prior_orders['eval_set'].astype('category')
prior_orders['product_name'] = prior_orders['product_name'].astype('category')

In [ ]:
train_orders.info()

In [ ]:
prior_orders.info()

In [ ]:
prior_orders.agg([min, max]).T

In [ ]:
train_orders.agg([min,max]).T

In [ ]:
user.agg([min,max]).T

In [ ]:
prior_orders.agg([min, max]).T

In [ ]:
prior_orders.describe()

In [ ]:
train_orders.describe()

**Step 4 : Data Cleaning**

In [ ]:
prior_orders.isnull().values.sum() 

In [ ]:
nas=pd.DataFrame(prior_orders.isnull().sum().sort_values(ascending=False)/len(prior_orders),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

In [ ]:
nas_t=pd.DataFrame(train_orders.isnull().sum().sort_values(ascending=False)/len(train_orders),columns = ['percent'])
pos_t = nas_t['percent'] > 0
nas_t[pos_t]